In [1]:
import pandas as pd
import numpy as np 
import sys
sys.path.append("../tax-calculator")
from taxcalc.records import Records
from taxcalc import Policy, Records, Calculator, Behavior, behavior
from taxcalc.utils import *
from numpy.testing import assert_array_almost_equal
import copy
import itertools

In [2]:
puf = pd.read_csv("../tax-calculator/puf.csv")

In [3]:
reform_proposal_cd = {
    2018: {'_ALD_StudentLoan_hc': [1],
           '_ALD_DomesticProduction_hc': [1],
           '_ALD_Tuition_hc': [1],
           "_DependentCredit_Child_c": [600],
           "_DependentCredit_Nonchild_c": [300],
           "_FilerCredit_c": [[300, 600, 300, 300, 300]],
           '_II_em':[0],
           '_STD': [[12000, 24000, 12000, 18000, 24000]],
           '_STD_Aged': [[0, 0, 0, 0, 0]],
           "_STD_Dep": [500],
           '_CR_SchR_hc': [1],
           '_ID_Medical_hc': [1],
           '_ID_StateLocalTax_hc': [1],
           '_ID_RealEstate_c': [[10000, 10000, 10000, 10000, 10000]],
           '_ID_Casualty_hc': [1],
           '_ID_Miscellaneous_hc': [1],
           '_ID_prt': [0],
           "_CG_brk1":[[38600, 77200, 38600, 51700, 77200]],
           "_CG_brk2":[[425800, 479000, 239500, 452400, 479000]],
           '_II_rt1': [0.12],
           '_II_brk1': [[45000, 90000, 45000, 67500, 90000]],
           '_II_rt2': [0.25],
           '_II_brk2': [[200000, 260000, 130000, 200000, 260000]],
           '_II_rt3': [0.35],
           '_II_brk3': [[500000, 1000000, 500000, 500000, 1000000]],
           '_II_rt4': [0.396],
           '_II_brk4': [[1000000, 1200000, 1000000, 1000000, 1200000]],
           '_II_rt5': [0.456],
           '_II_brk5': [[1207000, 1614000, 1207000, 1310500, 1614000]],
           '_II_rt6': [0.396],
           '_II_brk6': [[2000000, 2000000, 2000000, 2000000, 2000000]],
           '_II_rt7': [0.396],
           '_PT_rt1': [0.12],
           '_PT_brk1': [[45000, 90000, 45000, 67500, 90000]],
           '_PT_rt2': [0.25],
           '_PT_brk2': [[200000, 260000, 130000, 200000, 260000]],
           '_PT_rt3': [0.35],
           '_PT_brk3': [[500000, 1000000, 500000, 500000, 1000000]],
           '_PT_rt4': [0.396],
           '_PT_brk4': [[1000000, 1200000, 1000000, 1000000, 1200000]],
           '_PT_rt5': [0.456],
           '_PT_brk5': [[1207000, 1614000, 1207000, 1310500, 1614000]],
           '_PT_rt6': [0.396],
           '_PT_brk6': [[2000000, 2000000, 2000000, 2000000, 2000000]],
           '_PT_rt7': [0.396],
           '_AMT_rt1': [0],
           '_AMT_rt2': [0],
           "_LLC_Expense_c": [0]
        },
    2023: { 
            "_DependentCredit_Nonchild_c": [0],
            "_FilerCredit_c": [[0, 0, 0, 0, 0]],

        }
}

In [4]:
policy_base = Policy()
records_base = Records(puf)

policy_reform = Policy()
records_reform = Records(puf)
policy_reform.implement_reform(reform_proposal_cd)

calcbase = Calculator(policy = policy_base, records = records_base)
calcreform = Calculator(policy = policy_reform, records = records_reform)

/Users/seanwang/anaconda2/envs/taxcalc-dev/lib/python3.5/site-packages/numpy/core/numeric.py:294: FutureWarning: in the future, full((15,), 0) will return an array of dtype('int64')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)
/Users/seanwang/anaconda2/envs/taxcalc-dev/lib/python3.5/site-packages/numpy/core/numeric.py:294: FutureWarning: in the future, full((15,), 1) will return an array of dtype('int64')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)
/Users/seanwang/anaconda2/envs/taxcalc-dev/lib/python3.5/site-packages/numpy/core/numeric.py:294: FutureWarning: in the future, full((15, 5), 0) will return an array of dtype('int64')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)


You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.
You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.


In [5]:
calcreform.advance_to_year(2018)
calcbase.advance_to_year(2018)
calcbase.calc_all()
calcreform.calc_all()
assert calcbase.current_year == calcreform.current_year == 2018

In [6]:
adjusted_mea = calcbase.records.expanded_income/np.sqrt(calcbase.records.XTOT)
df = results(calcbase.records)
df['adjusted'] = adjusted_mea
df['age'] = calcbase.records.age_head
df['MARS'] = calcbase.records.MARS

In [7]:
df.sort_values('adjusted', inplace=True)
cumsum = df[(df.age<65)&(df.MARS!=1)].s006.cumsum()
cutoff = df[(df.age<65)&(df.MARS!=1)].s006.sum() / 2.0
median = df[(df.age<65)&(df.MARS!=1)].adjusted[cumsum >= cutoff].iloc[0]

In [8]:
median

37547.291359579642

In [9]:
n_df = results(calcbase.records)
n_df['adjusted'] = adjusted_mea
n_df['age'] = calcbase.records.age_head
n_df['MARS'] = calcbase.records.MARS

In [10]:
working_data_base = n_df[(n_df.adjusted>=(2/3)*median) & (n_df.adjusted<=(2)*median) & (n_df.age<65) & (n_df.MARS !=1)]

In [11]:
df_reform = results(calcreform.records)
df_reform['adjusted'] = calcreform.records.expanded_income/np.sqrt(calcreform.records.XTOT)
df_reform['age'] = calcreform.records.age_head
df_reform['MARS'] = calcreform.records.MARS
working_data_reform = df_reform.loc[working_data_base.index]

In [12]:
working_data_reform['tax_cut_flag'] = (working_data_base.combined > working_data_reform.combined)*1 + (working_data_base.combined == working_data_reform.combined)*0+ (working_data_base.combined < working_data_reform.combined)*(-1)

In [13]:
working_data_reform[working_data_reform.tax_cut_flag == 1].s006.sum()/working_data_reform.s006.sum()

0.72939970357582407

In [14]:
working_data_reform[working_data_reform.tax_cut_flag == -1].s006.sum()/working_data_reform.s006.sum()

0.2117223195537708

In [15]:
full_base = n_df
full_reform = df_reform
full_reform['tax_cut_flag'] = (full_base.combined > full_reform.combined)*1 + (full_base.combined == full_reform.combined)*0+ (full_base.combined < full_reform.combined)*(-1)

In [17]:
full_reform[full_reform.tax_cut_flag == 1].s006.sum()/full_reform.s006.sum()

0.47645641272187034

In [18]:
full_reform[full_reform.tax_cut_flag == -1].s006.sum()/full_reform.s006.sum()

0.19146052704737559

In [19]:
working_data_reform.s006.sum()/full_reform.s006.sum()

0.15660779504100775

In [20]:
working_data_base_itm = n_df[(n_df.adjusted>=(2/3)*median) & (n_df.adjusted<=(2)*median) & (n_df.age<65) & (n_df.MARS !=1)&(n_df.standard ==0)]

In [21]:
working_data_reform_itm = df_reform.loc[working_data_base_itm.index]

In [22]:
working_data_reform_itm['tax_cut_flag'] = (working_data_base_itm.combined > working_data_reform_itm.combined)*1 + (working_data_base_itm.combined == working_data_reform_itm.combined)*0+ (working_data_base_itm.combined < working_data_reform_itm.combined)*(-1)

In [23]:
working_data_reform_itm[working_data_reform_itm.tax_cut_flag == 1].s006.sum()/working_data_reform_itm.s006.sum()

0.50653266679775988

In [25]:
calcreform.advance_to_year(2026)
calcbase.advance_to_year(2026)
calcbase.calc_all()
calcreform.calc_all()
assert calcbase.current_year == calcreform.current_year == 2026

In [26]:
adjusted_mea = calcbase.records.expanded_income/np.sqrt(calcbase.records.XTOT)
df = results(calcbase.records)
df['adjusted'] = adjusted_mea
df['age'] = calcbase.records.age_head
df['MARS'] = calcbase.records.MARS

In [27]:
df.sort_values('adjusted', inplace=True)
cumsum = df[(df.age<65)&(df.MARS!=1)].s006.cumsum()
cutoff = df[(df.age<65)&(df.MARS!=1)].s006.sum() / 2.0
median = df[(df.age<65)&(df.MARS!=1)].adjusted[cumsum >= cutoff].iloc[0]

In [28]:
median

47550.085434022796

In [29]:
n_df = results(calcbase.records)
n_df['adjusted'] = adjusted_mea
n_df['age'] = calcbase.records.age_head
n_df['MARS'] = calcbase.records.MARS

In [30]:
working_data_base = n_df[(n_df.adjusted>=(2/3)*median) & (n_df.adjusted<=(2)*median) & (n_df.age<65) & (n_df.MARS !=1)]

In [31]:
df_reform = results(calcreform.records)
df_reform['adjusted'] = calcreform.records.expanded_income/np.sqrt(calcreform.records.XTOT)
df_reform['age'] = calcreform.records.age_head
df_reform['MARS'] = calcreform.records.MARS
working_data_reform = df_reform.loc[working_data_base.index]

In [34]:
working_data_reform['tax_cut_flag'] = (working_data_base.combined > working_data_reform.combined)*1 + (working_data_base.combined == working_data_reform.combined)*0+ (working_data_base.combined < working_data_reform.combined)*(-1)

In [35]:
working_data_reform[working_data_reform.tax_cut_flag == 1].s006.sum()/working_data_reform.s006.sum()

0.58202033493288075

In [36]:
working_data_reform[working_data_reform.tax_cut_flag == -1].s006.sum()/working_data_reform.s006.sum()

0.36000243889938593

In [37]:
full_base = n_df
full_reform = df_reform
full_reform['tax_cut_flag'] = (full_base.combined > full_reform.combined)*1 + (full_base.combined == full_reform.combined)*0+ (full_base.combined < full_reform.combined)*(-1)

In [38]:
full_reform[full_reform.tax_cut_flag == 1].s006.sum()/full_reform.s006.sum()

0.39391871761781339

In [41]:
full_reform[full_reform.tax_cut_flag == -1].s006.sum()/full_reform.s006.sum()

0.28571648834765828

In [40]:
working_data_base_itm = n_df[(n_df.adjusted>=(2/3)*median) & (n_df.adjusted<=(2)*median) & (n_df.age<65) & (n_df.MARS !=1)&(n_df.standard ==0)]

In [42]:
working_data_reform_itm = df_reform.loc[working_data_base_itm.index]

In [43]:
working_data_reform_itm['tax_cut_flag'] = (working_data_base_itm.combined > working_data_reform_itm.combined)*1 + (working_data_base_itm.combined == working_data_reform_itm.combined)*0+ (working_data_base_itm.combined < working_data_reform_itm.combined)*(-1)

In [44]:
working_data_reform_itm[working_data_reform_itm.tax_cut_flag == 1].s006.sum()/working_data_reform_itm.s006.sum()

0.25218402626493497